In [2]:
%load_ext autoreload
%autoreload 2

from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastai.vision.models.xresnet import *
from fastai.vision.models.unet import DynamicUnet
from bpho import *

/home/fredmonroe/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [5]:
from scipy.ndimage.interpolation import zoom as npzoom

def _micro_crappify(x, gauss_sigma = 1, poisson_loop=10, scale=4):
    x = x[0].numpy()
    x = x * 255.
    for n in range(poisson_loop):
        x = np.random.poisson(np.maximum(0,x).astype(np.int))
    x = x.astype(np.float32)
    noise = np.random.normal(0,gauss_sigma,size=x.shape).astype(np.float32)
    x = np.maximum(0,x+noise)
    x -= x.min()
    x /= x.max()
    return tensor(x[None])


class TfmEarly(Transform):
    "Decorator for pixel tfm funcs."
    order,_wrap = 0,'pixel'
micro_crappify = TfmEarly(_micro_crappify)




def get_src(x_data, y_data, size, zoom_scale=1.0, scale=1, skip = None, include=None):
    def map_to_hr(x):
        return y_data/x.relative_to(x_data)

    def blacklist(x):
        if skip is None: return True
        tag = x.stem.split('_')[0]
        return tag not in skip
    
    
    def whitelist(x):
        if include is None: return True
        tag = x.stem.split('_')[0]
        return tag in include
    
    #tfms = get_transforms(flip_vert=True, max_zoom=zoom_scale)
    tfms = [[],[]]
    y_tfms = [[t for t in tfms[0]], [t for t in tfms[1]]]
    #tfms[0].insert(0, micro_crappify())
    
    src = (NpyRawImageList.from_folder(x_data)
           .filter_by_func(blacklist)
           .filter_by_func(whitelist)
           .split_by_folder()
           .label_from_func(map_to_hr, label_cls=NpyRawImageList)
           .transform(tfms, size=size)
           .transform_y(y_tfms, size=size*scale))
    return src


def get_data(src, **kwargs):
    data = src.databunch(**kwargs)
    data.c = 3
    return data


In [6]:
x_data = Path('datasets/clean_001/lrup/')
y_data = Path('datasets/clean_001/hr')

src = get_src(x_data, y_data, size=128, zoom_scale=2, scale=1, include = ['mitotracker','actin'])
data = get_data(src, bs=4)
src

LabelLists;

Train: LabelList (1212 items)
x: NpyRawImageList
Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128)
y: NpyRawImageList
Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128)
Path: datasets/clean_001/lrup;

Valid: LabelList (21 items)
x: NpyRawImageList
Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128)
y: NpyRawImageList
Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128),Image (1, 128, 128)
Path: datasets/clean_001/lrup;

Test: None

In [7]:
model_dir = 'models'
arch = xresnet34
loss = F.l1_loss
metrics = sr_metrics
learn = xres_unet_learner(data, arch, path=Path('.'), loss_func=loss, metrics=metrics, model_dir=model_dir)


In [ ]:
learn.fit_one_cycle(1, 1e-4)

epoch,train_loss,valid_loss,ssim,psnr,time
